In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
dirWork         = os.getcwd()
dirData         = os.path.join(dirWork, r'data'        )
dirIntermediate = os.path.join(dirWork, r'intermediate')
dirResults      = os.path.join(dirWork, r'results'     )

print("Working Directory: "      + dirWork        )
print("Input Directory: "        + dirInput       )
print("Intermediate Directory: " + dirIntermediate)
print("Results Directory: "      + dirResults     )

Working Directory: e:\GitHub\Continuous-Count-Station-Data-Demo
Input Directory: e:\GitHub\Continuous-Count-Station-Data-Demo\input
Intermediate Directory: e:\GitHub\Continuous-Count-Station-Data-Demo\intermediate
Results Directory: e:\GitHub\Continuous-Count-Station-Data-Demo\results


In [5]:
df_CCSHourDirLane_Avenue = pd.read_csv(os.path.join(dirInput,'CCSHourDir_Combined_2013to2019.csv'))
display(df_CCSHourDirLane_Avenue)

FileNotFoundError: [Errno 2] No such file or directory: 'e:\\GitHub\\Continuous-Count-Station-Data-Demo\\input\\CCSHourDir_Combined_2013to2019.csv'

In [5]:
# initialize dataframe with avenue data
df_CCSHourDirLane = df_CCSHourDirLane_Avenue.copy()

#add 2019 Aug and no august data
df_CCSHourDirLane = df_CCSHourDirLane.append(df_CCSHourDirLane_OldFormat, ignore_index=True)
df_CCSHourDirLane = df_CCSHourDirLane.append(df_CCSHourDirLane_NewFormat, ignore_index=True)

df_CCSHourDirLane

,STATION,DIR,YEAR,MONTH,DAY,LANE,HOUR,HOURVOL,DOW
0,301,N,2013,1,1,0,0,358,1
1,301,N,2013,1,1,0,1,435,1
2,301,N,2013,1,1,0,2,277,1
3,301,N,2013,1,1,0,3,163,1
4,301,N,2013,1,1,0,4,129,1
...,...,...,...,...,...,...,...,...,...
17774744,716,N,2019,7,29,2,0,62,0
17774745,716,N,2019,7,29,3,0,112,0
17774746,716,P,2019,7,29,1,0,3,0
17774747,716,P,2019,7,29,2,0,42,0


In [6]:
df_CCSHourDirLane.set_index(['STATION','DIR','YEAR','MONTH','DAY','LANE','HOUR'])

HOURVOL  DOW
STATION DIR YEAR MONTH DAY LANE HOUR              
301     N   2013 1     1   0    0         358    1
                                1         435    1
                                2         277    1
                                3         163    1
                                4         129    1
...                                       ...  ...
716     N   2019 7     29  2    0          62    0
                           3    0         112    0
        P   2019 7     29  1    0           3    0
                           2    0          42    0
                           3    0          49    0

[17774749 rows x 2 columns]

In [7]:
df_CCSHourDir = df_CCSHourDirLane.groupby(['STATION','YEAR','MONTH','DAY','DOW','HOUR','DIR'],as_index=False).agg({'HOURVOL':[np.sum]})
df_CCSHourDir = df_CCSHourDir.droplevel(1, axis=1)
df_CCSHourDir

,STATION,YEAR,MONTH,DAY,DOW,HOUR,DIR,HOURVOL
0,301,2013,1,1,1,0,N,358
1,301,2013,1,1,1,0,P,288
2,301,2013,1,1,1,1,N,435
3,301,2013,1,1,1,1,P,292
4,301,2013,1,1,1,2,N,277
...,...,...,...,...,...,...,...,...
9379250,716,2019,6,29,5,0,P,7494
9379251,716,2019,6,30,6,0,N,5032
9379252,716,2019,6,30,6,0,P,4707
9379253,716,2019,7,29,0,0,N,7037


In [8]:
df_CCSHourDir.dtypes

STATION     int64
YEAR        int64
MONTH       int64
DAY         int64
DOW         int64
HOUR        int64
DIR        object
HOURVOL     int64
dtype: object

In [9]:
#for checks and understanding
#calculate number of days with data per station per month
df_1 = df_CCSHourDir.groupby(['STATION','MONTH','YEAR','DAY'],as_index=False).agg({'HOURVOL':[np.size]})
df_2 = df_1.groupby(['STATION','YEAR','MONTH'],as_index=False).agg({'DAY':[np.size]})
df_2 = df_2.droplevel(1, axis=1)
df_3 = df_2.pivot_table(index=['STATION'], columns=('YEAR','MONTH'), values='DAY').reset_index()

df_3 = df_3.fillna(0)
df_3 = df_3.astype(np.int32)

#export
df_3.to_csv(os.path.join(dirIntermediate, r'StationMonthDays.csv'),index=False)

In [10]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_3)
pd.set_option('display.max_rows', 10)

YEAR  STATION 2013                                             2014          \
MONTH            1   2   3   4   5   6   7   8   9  10  11  12    1   2   3   
0         301   31  28  30  30  31  28  28  31  30  22  30  31   31  28  30   
1         302   31  28  30  30  31  30  31  29  30  31  30  31   31  28  30   
2         303   23  27  30  30  31  30  31  31  30  31  30  31   31  28  30   
3         304   31  28  30  30  31  30  31  31  30  31  30  31   31  28  30   
4         305   28  28  30  30  28  30  25  31  30  31  30  31   31  22  30   
5         306   30  25  30  30  31  30  31  31  30  31  30  31   31  28  30   
6         307   31  28  30  30  31  30  31  31  30  31  30  27   31  20  30   
7         308   31  28  30  30  31  30  30  31  30  31  29  28   31  28  30   
8         309   25  28  30  30  31  30  30  31  30  31  30  27   31  28  29   
9         310   31  27  30  30  30  22  31  31  30  31  30  31   31  28  29   
10        312   23  28  28  30  18  30  31  31  30  31  30  31   30  28  29   
11        313   31  28  30  30  31  30  31  31  30  31  30  29   31  28  29   
12        314   30  28  30  30  31  30  31  30  30  31  11  31   31  28  30   
13        315   30  28  29  30  31  29  31  31  30  31  30  31   31  28  30   
14        316   31  28  30  30  31  30  31  31  30  31  30  31   31  28  30   
15        317   31  28  30  30  31  30  31  31  30  31  30  29   31  28  30   
16        318   27  28  30  30  31  30  31  31  30  31  30  26   31  28  29   
17        319   31  28  29  30  31  30  31  31  30  31  30  31   31  28  30   
18        320   24  28  30  30  22  18  27  30  26  29  30  28   31  25  30   
19        321   31  28  30  30  31  30  31  31  30  31  30  31   31  28  29   
20        322   31  28  30  30  31  30  27  30  30  25  30  31   31  28  30   
21        323   31  28  30  30  31  30  31  31  30  31  30  31   31  28  30   
22        324   31  28  30  30  31  30  31  31  30  31  30  31   31  28  30   
23        325   31  28  30  30  30  30  31  31  30  31  30  31   31  28  30   
24        327   31  28  29  30  31  30  31  31  30  31  30  31   30  28  30   
25        329   22  28  30  30  31  30  31  31  30  31  30  25   29  24  29   
26        332   31  25  30  30  22  30  31  31  30  15  23  27   19  28  26   
27        333   31  28  30  30  31  30  31  31  29  31  30  30   30  28  30   
28        335   31  28  30  30  31  30  29  28  30  27  30  29   30  28  30   
29        340   31  26  28  30  31  30  31  31  30  31  30  31   31  28  30   
30        341   31  28  30  30  31  29  30  31  30  31  30  31   31  28  29   
31        348   23  28  30  30  31  30  31  31  30  31  28  25   31  28  30   
32        349   31  28  30  30  30  30  31  31  30  31  29  31   31  28  30   
33        350   26  28  30  30  31  30  31  31  30  31  30  29   31  28  29   
34        351   31  28  30  30  31  30  31  31  30  31  30  31   31  28  30   
35        353   31  28  30  30  30  30  31  31  30  31  30  30   31  28  29   
36        354   31  28  30  30  31  30  31  31  29  31  30  30   31  28  30   
37        355   31  28  30  30  31  30  31  31  30  31  30  31   31  18  30   
38        362   31  28  30  30  31  30  31  31  30  31  30  31   31  28  30   
39        363   31  28  30  30  31  30  31  27  30  31  30  31   31  28  29   
40        382   31  28  30  30  31  30  31  31  26  31  30  31   31  24  26   
41        400   31  26  29  30  30  30  31  31  28  29  29  31   31  28  27   
42        401   31  28  30  30  31  30  31  31  30  28  28  25   12  27  29   
43        402   31  28  30  30  31  30  31  31  29  30  26  27   29  24  29   
44        403   31  27  30  27  31  30  31  31  26  31  29  30   31  28  28   
45        404   31  28  30  30  31  30  31  27  27  31  30  31   31  28  31   
46        405   31  28  30  30  31  30  31  31  30  31  30  31   31  23  21   
47        406   31  28  30  30  31  30  31  31  30  31  30  31   31  28  29   
48        407   31  28  30  25  31  30  31  31  30

In [11]:
#export to csv
df_CCSHourDir.to_csv(os.path.join(dirIntermediate, r'CCSHourDir_Combined_2013to2019.csv'),index=False)

In [12]:
#combine truck data
dfTruckDataAvenue = pd.read_csv(os.path.join(dirIntermediate,'AnnualTruckDailyTraffic_Avenue.csv'))
display(dfTruckDataAvenue)

dfTruckDataNewFormat = pd.read_csv(os.path.join(dirIntermediate,'AnnualTruckDailyTraffic_NewFormat.csv'))
display(dfTruckDataNewFormat)

# initialize dataframe with avenue data
dfTruckData = dfTruckDataAvenue.copy()

#add 2019 Aug and no august data
dfTruckData = dfTruckData.append(dfTruckDataNewFormat, ignore_index=True)

dfTruckData

,YEAR,ROUTE,BEGMP,ENDMP,DESC,AADT,SUTRK,CUTRK,SegID
0,2012,0006,0.00,46.01,Nevada State Line via SR 6 - Antelope Springs,325.0,0.24,0.23,000600000
1,2012,0006,46.01,77.54,Antelope Springs via SR 6 - Gunnison Massacre ...,340.0,0.17,0.33,000604602
2,2012,0006,77.54,82.88,Gunnison Massacre Site via SR 6 - Main St Hin...,420.0,0.16,0.26,000607754
3,2012,0006,82.88,83.89,Main St via SR 6 (500 N) Hinckley - SR 257 (40...,1570.0,0.14,0.19,000608288
4,2012,0006,83.89,87.68,SR 257 (4000 W) Hinckley via SR 6 - 1000 W to ...,3270.0,0.13,0.12,000608390
...,...,...,...,...,...,...,...,...,...
9321,2016,0320,0.00,2.19,Emergency Vehicle Operation Facility Camp Will...,160.0,0.10,0.29,032000000
9322,2016,0491,0.00,0.44,SR 191 (Main St) via Center St - 500 E Monticello,5400.0,0.05,0.24,049100000
9323,2016,0491,0.44,2.01,500 E via Center St - Port of Entry Monticello,3000.0,0.05,0.24,049100044
9324,2016,0491,2.01,17.02,Monticello POE via SR 491 - Colorado State Lin...,2800.0,0.05,0.24,049100202


,YEAR,ROUTE,BEGMP,ENDMP,DESC,AADT,SUTRK,CUTRK,SegID
0,2017,0006,0.000,46.017,Nevada State Line via SR 6 - Antelope Springs,409.0,0.249610,0.232449,000600000
1,2017,0006,46.017,77.545,Antelope Springs via SR 6 - Gunnison Massacre ...,366.0,0.175063,0.333819,000604602
2,2017,0006,77.545,82.885,Gunnison Massacre Site via SR 6 - Main St Hinc...,525.0,0.162481,0.264722,000607755
3,2017,0006,82.885,83.897,Main St via SR 6 (500 N) Hinckley - SR 257 (40...,1961.0,0.149898,0.195620,000608289
4,2017,0006,83.897,87.687,SR 257 (4000 W) Hinckley via SR 6 - 1000 W to ...,3468.0,0.137314,0.126513,000608390
...,...,...,...,...,...,...,...,...,...
5608,2019,0319,0.000,0.901,SR 40 Mayflower - Jordanelle Reservoir Boat Ramp,1287.0,0.094570,0.254773,031900000
5609,2019,0491,0.000,0.456,SR 191 (Main St) via Center St - 500 E Monticello,5741.0,0.070659,0.297167,049100000
5610,2019,0491,0.456,2.031,500 E via Center St - Port of Entry Monticello,3313.0,0.070659,0.297167,049100046
5611,2019,0491,2.031,17.069,Monticello POE via SR 491 - Colorado State Lin...,3092.0,0.070659,0.297167,049100203


,YEAR,ROUTE,BEGMP,ENDMP,DESC,AADT,SUTRK,CUTRK,SegID
0,2012,0006,0.000,46.010,Nevada State Line via SR 6 - Antelope Springs,325.0,0.240000,0.230000,000600000
1,2012,0006,46.010,77.540,Antelope Springs via SR 6 - Gunnison Massacre ...,340.0,0.170000,0.330000,000604602
2,2012,0006,77.540,82.880,Gunnison Massacre Site via SR 6 - Main St Hin...,420.0,0.160000,0.260000,000607754
3,2012,0006,82.880,83.890,Main St via SR 6 (500 N) Hinckley - SR 257 (40...,1570.0,0.140000,0.190000,000608288
4,2012,0006,83.890,87.680,SR 257 (4000 W) Hinckley via SR 6 - 1000 W to ...,3270.0,0.130000,0.120000,000608390
...,...,...,...,...,...,...,...,...,...
14934,2019,0319,0.000,0.901,SR 40 Mayflower - Jordanelle Reservoir Boat Ramp,1287.0,0.094570,0.254773,031900000
14935,2019,0491,0.000,0.456,SR 191 (Main St) via Center St - 500 E Monticello,5741.0,0.070659,0.297167,049100000
14936,2019,0491,0.456,2.031,500 E via Center St - Port of Entry Monticello,3313.0,0.070659,0.297167,049100046
14937,2019,0491,2.031,17.069,Monticello POE via SR 491 - Colorado State Lin...,3092.0,0.070659,0.297167,049100203


In [13]:
import csv
dfTruckData.to_csv(os.path.join(dirIntermediate, r'TruckData_Combined_2012to2019.csv'),index=False,quotechar='"', quoting=csv.QUOTE_NONNUMERIC)

In [16]:
dfTruckData[(dfTruckData['ROUTE']=='0175') & (dfTruckData['YEAR']==2019)]

,YEAR,ROUTE,BEGMP,ENDMP,DESC,AADT,SUTRK,CUTRK,SegID
14566,2019,0175,0.000,2.400,Jct SR 154 via 11400 South,32668.0,0.251732,0.037031,017500000
14567,2019,0175,2.400,2.905,SR 68,20166.0,0.251732,0.037031,017500240
14568,2019,0175,2.905,3.892,1300 West,22213.0,0.251732,0.037031,017500291
14569,2019,0175,3.892,4.905,700 West - SR 89,22406.0,0.251732,0.037031,017500389
